In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<p style = "font-size : 50px; color : blue ; font-family : 'Comic Sans MS'; text-align : center; background-color : yellow; border-radius: 5px 5px;"><strong>Titanic EDA and Prediction</strong></p>

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Importing all necessary libraries</strong></p> 

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
pd.pandas.set_option("display.max_columns",None)
print("all necessary libraries are imported")

In [ ]:
train=pd.read_csv("../input/titanic/train.csv")
train.head()

In [ ]:
test=pd.read_csv("../input/titanic/test.csv")
test.head()

In [ ]:
train.shape,test.shape

In [ ]:
train_cat=list(train.select_dtypes(include='object'))
train_cat


In [ ]:
train_num=list(train.select_dtypes(exclude='object'))
train_num

<p style = "font-size : 35px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Univariate Analysis</strong></p> 

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Target Feature</strong></p> 

In [ ]:
train['Survived'].value_counts().plot.bar(color=['r','g'])

it says that most of the people have dead

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Categorical Feature</strong></p> 

In [ ]:
plt.figure(1)
plt.subplot(221)
train['Sex'].value_counts(normalize=True).plot.bar(figsize=(10,10),color=['b','r'])
plt.subplot(223)
train['Embarked'].value_counts(normalize=True).plot.bar(figsize=(10,10),color=['y','g','r'])

* > Around 80% of the passangers are male
* > Around 75% of passangers have embarked as S

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Numerical Feature</strong></p> 

In [ ]:
plt.figure(1)
plt.subplot(221)
train['Pclass'].value_counts(normalize=True).plot.bar(figsize=(10,10),title="Pclass")
plt.subplot(223)
train['Pclass'].plot.box(figsize=(10,10))

In [ ]:
plt.figure(1)
plt.subplot(221)
train['SibSp'].value_counts(normalize=True).plot.bar(figsize=(10,10),title="SibSp")
plt.subplot(223)
train['SibSp'].plot.box(figsize=(10,10))

In [ ]:
plt.figure(1)
plt.subplot(221)
train['Parch'].value_counts(normalize=True).plot.bar(figsize=(10,10),title="Parch")
plt.subplot(223)
train['Parch'].plot.box(figsize=(10,10))

<p style = "font-size : 35px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Bivariate Analysis</strong></p> 

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Categorical vs Target Feature</strong></p> 

In [ ]:
train_cat

In [ ]:
sex=pd.crosstab(train['Sex'],train['Survived'])
sex.div(sex.sum(1).astype(float),axis=0).plot(kind='bar',stacked=True,figsize=(4,4))

> More males have dead comapre to females
as if 1 shows alive and 0 shows death

In [ ]:
embarked=pd.crosstab(train['Embarked'],train['Survived'])
embarked.div(embarked.sum(1).astype(float),axis=0).plot(kind='bar',stacked=True,figsize=(4,4))

> The survival rate of embarked 'C' is greater than the rest two and the lowest survival rate is associated with embarked 'S'

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Numerical vs Target Feature</strong></p> 

In [ ]:
train_num

In [ ]:
pclass=pd.crosstab(train['Pclass'],train['Survived'])
pclass.div(pclass.sum(1).astype(float),axis=0).plot(kind='bar',stacked=True,figsize=(4,4))

> class 1 has most survival rate and class 3 has least survival rate

In [ ]:
sibsp=pd.crosstab(train['SibSp'],train['Survived'])
sibsp.div(sibsp.sum(1).astype(float),axis=0).plot(kind='bar',stacked=True,figsize=(4,4))

In [ ]:
parch=pd.crosstab(train['Parch'],train['Survived'])
parch.div(parch.sum(1).astype(float),axis=0).plot(kind='bar',stacked=True,figsize=(4,4))

In [ ]:
train.isnull().any()

In [ ]:
train_missing_obj_col=[]
for col in train_cat:
    if train[col].isnull().any():
        train_missing_obj_col.append(col)
train_missing_obj_col        

<p style = "font-size : 35px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Missing Values Imputation In Train Dataset</strong></p>

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Imputation In Numerical Col</strong></p> 

In [ ]:
train_missing_num_col=[]
for col in train_num:
    if train[col].isnull().any():
        train_missing_num_col.append(col)
train_missing_num_col     

In [ ]:
temp=train[['Age']]
temp

In [ ]:
train.drop(['Age'],inplace=True,axis=1)

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer=SimpleImputer()
imputed_temp = pd.DataFrame(my_imputer.fit_transform(temp))
imputed_temp.columns = temp.columns

In [ ]:
train=pd.concat([train,imputed_temp],axis=1)

In [ ]:
train

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Imputation In Categorical Col</strong></p>

In [ ]:
train['Embarked'].fillna(train['Embarked'].mode(),inplace=True)

In [ ]:
dummy1=pd.get_dummies(train[['Sex','Embarked']],drop_first=True)
dummy1

In [ ]:
train.drop(train_missing_obj_col ,axis=1,inplace=True)

In [ ]:
train=pd.concat([train,dummy1],axis=1)

In [ ]:
train.drop(['Name','PassengerId','Sex','Ticket'],axis=1,inplace=True)

In [ ]:
train

In [ ]:
train.shape

<p style = "font-size : 35px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Removing the Skewness In the Train Data Col</strong></p>

In [ ]:
train['Age']=np.log(train['Age']+1)

In [ ]:
train['Fare']=np.log(train['Fare']+1)

In [ ]:
 (train['Age']).plot(kind = 'density', title = 'Log Age distribution', fontsize=14, figsize=(10, 6))

In [ ]:
 (train['Survived']).plot(kind = 'density', title = 'Survived', fontsize=14, figsize=(10, 6))

<p style = "font-size : 35px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Feature Engineering on Test Dataset</strong></p>

In [ ]:
test.head()

In [ ]:
test_cat_col=list(test.select_dtypes(include='object'))
test_cat_col

In [ ]:
test_num_col=list(test.select_dtypes(exclude='object'))
test_num_col

In [ ]:
test_missing_cat_col=[col for col in test_cat_col if test[col].isnull().any()]
test_missing_cat_col

In [ ]:
test_missing_num_col=[col for col in test_num_col if test[col].isnull().any()]
test_missing_num_col

<p style = "font-size : 35px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Missing Values Imputation In Test Dataset</strong></p>

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Imputation In Numerical Col</strong></p>

In [ ]:
temp1=test[['Age','Fare']]

In [ ]:
imputer=SimpleImputer()
imputed_temp1=pd.DataFrame(imputer.fit_transform(temp1))
imputed_temp1.columns=temp1.columns

In [ ]:
test.drop(['Age','Cabin','Name','Fare'],axis=1,inplace=True)

In [ ]:
test=pd.concat([test,imputed_temp1],axis=1)
test

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Imputation In Categorical Col</strong></p>

In [ ]:
dummy2=pd.get_dummies(test[['Sex','Embarked']],drop_first=True)
dummy2

In [ ]:
test.drop(['PassengerId','Sex','Embarked','Ticket'],axis=1,inplace=True)

In [ ]:
test=pd.concat([test,dummy2],axis=1)
test

In [ ]:
test.shape

<p style = "font-size : 35px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Removing the Skewness In the Test Data Col</strong></p>

In [ ]:
test['Age']=np.log(test['Age'])

In [ ]:
test['Fare']=np.log(test['Fare']+1)

In [ ]:
 (test['Age']).plot(kind = 'density', title = 'Log Age distribution', fontsize=14, figsize=(10, 6))

<p style = "font-size : 20px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Removing Target Variable from the Train Dataset</strong></p>

In [ ]:
y=train['Survived']

In [ ]:
train.drop(['Survived'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape,test.shape

<p style = "font-size : 35px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Model Building</strong></p>

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Logistic Regression</strong></p>

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(train,y,random_state=42)

In [ ]:
logistic=LogisticRegression(max_iter=100,random_state=1,n_jobs=-1)
logistic.fit(x_train,y_train)
pred1=logistic.predict(x_test)
pred1

In [ ]:
print(f'The train accuracy is {logistic.score(x_train,y_train)}')
from sklearn.metrics import accuracy_score
print('Logistic Regresson model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred1)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Random Forest Classifier</strong></p>

# Hyperparameter tuning

In [ ]:
# param_grid = { "criterion" : ["gini","entropy"], "min_samples_leaf" : [1, 5, 10, 25, 50, 70], 
#               "min_samples_split" : [2, 4, 10, 12, 16, 18, 25, 35], "n_estimators": [100, 400, 700, 1000, 1500]}
# rf = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1, n_jobs=-1)
# clf = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1)
# clf.fit(x_train, y_train)
# clf.best_params_

# Now buliding the model with best parameters

In [ ]:
random_forest = RandomForestClassifier(criterion = "entropy", 
                                       min_samples_leaf = 1, 
                                       min_samples_split = 12,
                                       n_estimators=100, 
                                       max_features='auto', 
                                       oob_score=True, 
                                       random_state=42, 
                                       n_jobs=-1)

random_forest.fit(x_train, y_train)
pred2 = random_forest.predict(x_test)

random_forest.score(x_train, y_train)

print("oob score:", round(random_forest.oob_score_, 4)*100, "%")

In [ ]:
print('Random Forest Classifier model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred2)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong> Stochastic Gradient Descent (SGD)</strong></p>

In [ ]:
sgd = SGDClassifier(random_state=42)
sgd.fit(x_train, y_train)
pred3 = sgd.predict(x_test)

sgd.score(x_train, y_train)

acc_sgd = round(sgd.score(x_train, y_train) * 100, 2)
acc_sgd

In [ ]:
print('Stochastic Gradient Descent (SGD) model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred3)))

# Hyperparameter tuning

In [ ]:

sgdc=SGDClassifier(random_state=0)

parameters = {
            'alpha':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 
             'loss':['hinge','log'], 'penalty':['l1','l2']}
searcher = GridSearchCV(sgdc, parameters, cv=10)
searcher.fit(x_train, y_train)
print(searcher.best_params_)
print(searcher.best_score_)

# Now buliding the model with best parameters

In [ ]:
sgd = SGDClassifier(random_state=0,alpha=0.01,loss='log',penalty='l2')
sgd.fit(x_train, y_train)
pred3 = sgd.predict(x_test)
pred3

In [ ]:
print('Stochastic Gradient Descent (SGD) tuned model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred3)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong> K Nearest Neighbor (KNN)</strong></p>

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train, y_train) 
pred4 = knn.predict(x_test)  
acc_knn = round(knn.score(x_train, y_train) * 100, 2)
print('Stochastic Gradient Descent (SGD) model train accuracy score: {0:0.4f}'. format(acc_knn))
print('Stochastic Gradient Descent (SGD) model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred4)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Linear Support Vector Machine</strong></p># Linear Support Vector Machine

In [ ]:
linear_svc = LinearSVC(max_iter=100000,dual=True)
linear_svc.fit(x_train, y_train)

pred5 = linear_svc.predict(x_test)

acc_linear_svc = round(linear_svc.score(x_train, y_train) * 100, 2)
print('Linear Support Vector Machine model train accuracy score: {0:0.4f}'. format(acc_linear_svc))
print('Linear Support Vector Machine model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred5)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Decision Tree</strong></p>

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
pred6 = decision_tree.predict(x_test)
acc_decision_tree = round(decision_tree.score(x_train,y_train) * 100, 2)
print('Decision Tree model train accuracy score: {0:0.4f}'. format(acc_decision_tree))
print('Decision Tree model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred6)))

# **Hyperparameter tuning**

In [ ]:
grid_param = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [3, 5, 7, 10],
    'splitter' : ['best', 'random'],
    'min_samples_leaf' : [1, 2, 3, 5, 7],
    'min_samples_split' : [1, 2, 3, 5, 7],
    'max_features' : ['auto', 'sqrt', 'log2']
}

decision = GridSearchCV(decision_tree, grid_param, cv = 5, n_jobs = -1, verbose = 1)
decision.fit(x_train, y_train)
print(decision.best_params_)
print(decision.best_score_)

# Now buliding the model with best parameters

In [ ]:
decision_tree = DecisionTreeClassifier(criterion='entropy',
                                      max_depth=7,
                                      splitter='best',
                                      min_samples_leaf=5,
                                      min_samples_split=3,
                                      max_features='log2')
decision_tree.fit(x_train, y_train)
pred_6 = decision_tree.predict(x_test)
print('Decision Tree model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred_6)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Adaboost Classifier</strong></p>

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
adb = AdaBoostClassifier(base_estimator = decision_tree)
adb.fit(x_train,y_train)
adb_pred=adb.predict(x_test)
acc_adb=adb.score(x_train,y_train)
print('Adaboost Classifier model train accuracy score: {0:0.4f}'. format(acc_adb))
print('Adaboost Classifier model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, adb_pred)))

# Hyperparameter Tuning

In [ ]:
grid_param = {
    'n_estimators' : [100, 120, 150, 180, 200],
    'learning_rate' : [0.01, 0.1, 1, 10],
    'algorithm' : ['SAMME', 'SAMME.R']
}

ada = GridSearchCV(adb, grid_param, cv = 5, n_jobs = -1, verbose = True)
ada.fit(x_train, y_train)
print(ada.best_params_)
print(ada.best_score_)

In [ ]:
adb = AdaBoostClassifier(n_estimators=200,
                        learning_rate=0.01,
                        algorithm='SAMME.R')
adb.fit(x_train,y_train)
pred7=adb.predict(x_test)
print('Adaboost Classifier model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred7)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Gradient Boosting</strong></p>

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(x_train, y_train)
pred8=gb.predict(x_test)
acc_gb=gb.score(x_train,y_train)
print('Gradient Boosting model train accuracy score: {0:0.4f}'. format(acc_gb))
print('Gradient Boosting model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred8)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>LightGBM</strong></p>

In [ ]:
import lightgbm as lgb
lgbm= lgb.LGBMClassifier()
lgbm.fit(x_train,y_train)
lgbm_pred=lgbm.predict(x_test)
acc_lgbm=lgbm.score(x_train,y_train)
print('lgbm model train accuracy score: {0:0.4f}'. format(acc_lgbm))
print('lgbm model accuracy score: {0:0.4f}'. format(accuracy_score(y_test,lgbm_pred)))

# Hyperparameter Tuning

In [ ]:
fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'auc', 
            "eval_set" : [(x_test,y_test)],
            'eval_names': ['valid'],
            'verbose': 100,
            'categorical_feature': 'auto'}

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(50, 100), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
clf = lgb.LGBMClassifier(max_depth=-1, random_state=42, silent=True, metric='None', n_jobs=4, n_estimators=100)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=100,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

In [ ]:
gs.fit(x_train, y_train, **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

In [ ]:
opt_params={'colsample_bytree':  0.952164731370897, 'min_child_samples': 61, 
            'min_child_weight': 0.01, 'num_leaves': 38, 'reg_alpha': 0, 
            'reg_lambda': 0.1, 'subsample':0.3029313662262354} 

In [ ]:
clf_sw = lgb.LGBMClassifier(**clf.get_params())
#set optimal parameters
clf_sw.set_params(**opt_params)

In [ ]:
clf_sw.fit(x_train,y_train)
pred9=clf_sw.predict(x_test)
print('lgbm model accuracy score: {0:0.4f}'. format(accuracy_score(y_test,pred9)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>CatBoost Classifier</strong></p>

In [ ]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(iterations=10)
cat.fit(x_train, y_train)
pred10=cat.predict(x_test)
acc_cat=cat.score(x_train,y_train)
print('CatBoostClassifier model accuracy score: {0:0.4f}'. format(accuracy_score(y_test,pred10)))

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Voting Classifier</strong></p>

In [ ]:
from sklearn.ensemble import VotingClassifier 
classifiers = [('Gradient Boosting Classifier', gb), ('Stochastic Gradient Boosting', sgd),  ('Cat Boost Classifier', cat), 
                ('Decision Tree', decision_tree), ('Light Gradient', clf_sw),
               ('Random Forest', random_forest), ('Ada Boost', adb), ('Logistic', logistic)]
vc = VotingClassifier(estimators = classifiers)
vc.fit(x_train, y_train) 
voting_pred = vc.predict(x_test)
accuracy = accuracy_score(y_test,voting_pred)
print('Voting Classifier: {:.3f}'.format(accuracy))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(f"{confusion_matrix(y_test, vc.predict(x_test))}\n")
print(classification_report(y_test, vc.predict(x_test)))

<p style = "font-size : 35px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Important Features</strong></p>

In [ ]:
importances = pd.DataFrame({'feature':train.columns,'importance':np.round(random_forest.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances.head(15)

In [ ]:
importances.plot.bar()

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Model Comaprison</strong></p>

In [ ]:
models = pd.DataFrame({
    'Model' : ['Logistic Regression','Random Forest Classifier','Stochastic Gradient Boosting', 'KNN','LSVM', 'Decision Tree Classifier','Ada Boost Classifier',
             'Gradient Boosting Classifier','Lightgbm', 'Cat Boost',  'Voting Classifier'],
    'Score' : [accuracy_score(y_test,pred1), accuracy_score(y_test,pred2), accuracy_score(y_test,pred3),
               accuracy_score(y_test,pred4),accuracy_score(y_test,pred5), accuracy_score(y_test,pred_6),
               accuracy_score(y_test,pred7), accuracy_score(y_test,pred8), accuracy_score(y_test,pred9)
               ,accuracy_score(y_test,pred10),accuracy]
})


models.sort_values(by = 'Score', ascending = False)

<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Final Prediction</strong></p>

In [ ]:
final_pred=vc.predict(test)

In [ ]:
submission=pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
submission

In [ ]:
submission['Survived']=final_pred

In [ ]:
submission

In [ ]:
pd.DataFrame(submission,columns=['PassengerId','Survived']).to_csv('titanic_new.csv',index=False)

In [ ]:
pd.read_csv('titanic_new.csv')